In [1]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2

In [3]:
import pandas as pd

df = pd.read_csv('flowers/train_labels.csv')

df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,_IGP1402.jpg,4928,3264,flower,3444,1790,4151,2783
1,_IGP1403.jpg,4928,3264,flower,2419,912,3094,1622
2,_IGP1403.jpg,4928,3264,flower,4062,1515,4537,2183
3,_IGP1404.jpg,4928,3264,flower,2708,1608,2958,1854
4,_IGP1405.jpg,4928,3264,flower,3301,401,4140,1129


In [8]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode
import itertools
import cv2

def get_tomato_dicts(csv_file, img_dir):
    df = pd.read_csv(csv_file)
    df['filename'] = df['filename'].map(lambda x: img_dir+x)

    classes = ['flower']
    #df['class'] = df['class'].map(lambda x: '')
    df['class_int'] = df['class'].map(lambda x: classes.index(x))

    dataset_dicts = []
    for filename in df['filename'].unique().tolist():
        record = {}
        
        image = cv2.imread(filename)
        
        height, width, _ = image.shape
        
        record["file_name"] = filename
        record["height"] = height
        record["width"] = width

        objs = []
        for index, row in df[(df['filename']==filename)].iterrows():
          obj= {
              'bbox': [row['xmin'], row['ymin'], row['xmax'], row['ymax']],
              'bbox_mode': BoxMode.XYXY_ABS,
              'category_id': row['class_int'],
              "iscrowd": 0
          }
          objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [10]:
from detectron2.data import DatasetCatalog, MetadataCatalog

classes = ['flower']

for d in ["train", "test"]:
  DatasetCatalog.register('flower2_data/' + d, lambda d=d: get_tomato_dicts('flowers/' + d + '_labels.csv', 'flowers/' + d+'/'))
  MetadataCatalog.get('flower2_data/' + d).set(thing_classes=classes)
tomato_metadata = MetadataCatalog.get('flower2_data/train')

In [11]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.logger import setup_logger
setup_logger()
cfg = get_cfg()
cfg.merge_from_file("./detectron2/model_zoo/configs/COCO-Detection/retinanet_R_101_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = "model_final_59f53c.pkl"
cfg.DATASETS.TRAIN = ('flower2_data/train',)
cfg.DATASETS.TEST = ()   
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.MAX_ITER = 10000
cfg.MODEL.RETINANET.NUM_CLASSES = 1
cfg.SOLVER.BASE_LR = 0.0001
cfg.MODEL.DEVICE = "cuda:0"
cfg.OUTPUT_DIR = 'output_flower'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

Loading config ./detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


WARNING [01/03 21:56:55 d2.config.compat]: Config './detectron2/model_zoo/configs/COCO-Detection/retinanet_R_101_FPN_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
[01/03 21:56:56 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (co

[01/03 21:57:20 d2.data.build]: Removed 0 images with no usable annotations. 90 images left.
[01/03 21:57:20 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|   flower   | 104          |
|            |              |
[01/03 21:57:20 d2.data.detection_utils]: TransformGens used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[01/03 21:57:20 d2.data.build]: Using training sampler TrainingSampler


/home/analiza_obrazu/anaconda3/envs/triffid/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/analiza_obrazu/anaconda3/envs/triffid/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/analiza_obrazu/anaconda3/envs/triffid/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

[01/03 21:57:20 d2.engine.train_loop]: Starting training from iteration 0
[01/03 21:57:50 d2.utils.events]: eta: 3:26:01  iter: 19  total_loss: 1.646  loss_cls: 1.166  loss_box_reg: 0.470  time: 1.4149  data_time: 0.6478  lr: 0.000002  max_mem: 5584M
[01/03 21:58:19 d2.utils.events]: eta: 3:25:36  iter: 39  total_loss: 1.636  loss_cls: 1.165  loss_box_reg: 0.486  time: 1.4198  data_time: 0.5627  lr: 0.000004  max_mem: 5584M
[01/03 21:58:47 d2.utils.events]: eta: 3:25:25  iter: 59  total_loss: 1.589  loss_cls: 1.153  loss_box_reg: 0.442  time: 1.4110  data_time: 0.5613  lr: 0.000006  max_mem: 5584M
[01/03 21:59:15 d2.utils.events]: eta: 3:25:10  iter: 79  total_loss: 1.550  loss_cls: 1.138  loss_box_reg: 0.420  time: 1.4149  data_time: 0.5703  lr: 0.000008  max_mem: 5584M
[01/03 21:59:43 d2.utils.events]: eta: 3:26:58  iter: 99  total_loss: 1.521  loss_cls: 1.117  loss_box_reg: 0.389  time: 1.4088  data_time: 0.5834  lr: 0.000010  max_mem: 5584M
[01/03 22:00:12 d2.utils.events]: eta: 3:

...

[01/04 00:48:19 d2.utils.events]: eta: 1:11:45  iter: 7059  total_loss: 0.004  loss_cls: 0.001  loss_box_reg: 0.003  time: 1.4517  data_time: 0.6113  lr: 0.000100  max_mem: 5584M
[01/04 00:48:48 d2.utils.events]: eta: 1:11:16  iter: 7079  total_loss: 0.004  loss_cls: 0.001  loss_box_reg: 0.003  time: 1.4518  data_time: 0.6335  lr: 0.000100  max_mem: 5584M
[01/04 00:49:18 d2.utils.events]: eta: 1:10:47  iter: 7099  total_loss: 0.004  loss_cls: 0.001  loss_box_reg: 0.003  time: 1.4518  data_time: 0.6401  lr: 0.000100  max_mem: 5584M
[01/04 00:49:47 d2.utils.events]: eta: 1:10:11  iter: 7119  total_loss: 0.004  loss_cls: 0.001  loss_box_reg: 0.003  time: 1.4518  data_time: 0.7056  lr: 0.000100  max_mem: 5584M
[01/04 00:50:16 d2.utils.events]: eta: 1:09:48  iter: 7139  total_loss: 0.004  loss_cls: 0.001  loss_box_reg: 0.003  time: 1.4519  data_time: 0.7018  lr: 0.000100  max_mem: 5584M
[01/04 00:50:45 d2.utils.events]: eta: 1:09:19  iter: 7159  total_loss: 0.004  loss_cls: 0.001  loss_box_

[01/04 01:09:41 d2.utils.events]: eta: 0:49:28  iter: 7939  total_loss: 0.004  loss_cls: 0.001  loss_box_reg: 0.003  time: 1.4522  data_time: 0.6081  lr: 0.000100  max_mem: 5584M
[01/04 01:10:10 d2.utils.events]: eta: 0:49:00  iter: 7959  total_loss: 0.003  loss_cls: 0.001  loss_box_reg: 0.003  time: 1.4522  data_time: 0.6175  lr: 0.000100  max_mem: 5584M
[01/04 01:10:40 d2.utils.events]: eta: 0:48:31  iter: 7979  total_loss: 0.003  loss_cls: 0.001  loss_box_reg: 0.003  time: 1.4523  data_time: 0.6297  lr: 0.000100  max_mem: 5584M
[01/04 01:11:08 d2.utils.events]: eta: 0:48:02  iter: 7999  total_loss: 0.004  loss_cls: 0.001  loss_box_reg: 0.003  time: 1.4522  data_time: 0.5637  lr: 0.000100  max_mem: 5584M
[01/04 01:11:37 d2.utils.events]: eta: 0:47:33  iter: 8019  total_loss: 0.004  loss_cls: 0.001  loss_box_reg: 0.003  time: 1.4521  data_time: 0.5721  lr: 0.000100  max_mem: 5584M
[01/04 01:12:06 d2.utils.events]: eta: 0:47:04  iter: 8039  total_loss: 0.004  loss_cls: 0.001  loss_box_

[01/04 01:31:02 d2.utils.events]: eta: 0:28:00  iter: 8819  total_loss: 0.004  loss_cls: 0.001  loss_box_reg: 0.003  time: 1.4524  data_time: 0.6207  lr: 0.000100  max_mem: 5584M
[01/04 01:31:30 d2.utils.events]: eta: 0:27:07  iter: 8839  total_loss: 0.003  loss_cls: 0.001  loss_box_reg: 0.002  time: 1.4523  data_time: 0.5547  lr: 0.000100  max_mem: 5584M
[01/04 01:31:59 d2.utils.events]: eta: 0:26:39  iter: 8859  total_loss: 0.003  loss_cls: 0.000  loss_box_reg: 0.002  time: 1.4523  data_time: 0.6170  lr: 0.000100  max_mem: 5584M
[01/04 01:32:28 d2.utils.events]: eta: 0:26:11  iter: 8879  total_loss: 0.003  loss_cls: 0.001  loss_box_reg: 0.002  time: 1.4523  data_time: 0.6099  lr: 0.000100  max_mem: 5584M
[01/04 01:32:58 d2.utils.events]: eta: 0:25:43  iter: 8899  total_loss: 0.003  loss_cls: 0.001  loss_box_reg: 0.002  time: 1.4524  data_time: 0.6117  lr: 0.000100  max_mem: 5584M
[01/04 01:33:27 d2.utils.events]: eta: 0:25:15  iter: 8919  total_loss: 0.003  loss_cls: 0.001  loss_box_

[01/04 01:52:20 d2.utils.events]: eta: 0:06:26  iter: 9699  total_loss: 0.003  loss_cls: 0.001  loss_box_reg: 0.002  time: 1.4524  data_time: 0.6294  lr: 0.000100  max_mem: 5584M
[01/04 01:52:49 d2.utils.events]: eta: 0:06:00  iter: 9719  total_loss: 0.003  loss_cls: 0.000  loss_box_reg: 0.002  time: 1.4523  data_time: 0.6330  lr: 0.000100  max_mem: 5584M
[01/04 01:53:18 d2.utils.events]: eta: 0:05:34  iter: 9739  total_loss: 0.003  loss_cls: 0.001  loss_box_reg: 0.002  time: 1.4523  data_time: 0.6159  lr: 0.000100  max_mem: 5584M
[01/04 01:53:47 d2.utils.events]: eta: 0:05:48  iter: 9759  total_loss: 0.002  loss_cls: 0.000  loss_box_reg: 0.002  time: 1.4523  data_time: 0.7198  lr: 0.000100  max_mem: 5584M
[01/04 01:54:16 d2.utils.events]: eta: 0:05:19  iter: 9779  total_loss: 0.003  loss_cls: 0.000  loss_box_reg: 0.002  time: 1.4523  data_time: 0.6854  lr: 0.000100  max_mem: 5584M
[01/04 01:54:46 d2.utils.events]: eta: 0:04:50  iter: 9799  total_loss: 0.003  loss_cls: 0.000  loss_box_

OrderedDict()